<center> <h1>Homework 2</h1> </center>
<p style="text-align: center;">author: EL FATTAHI Charaf-Ed-Dine</p>

In [1]:
using JuMP

using LinearAlgebra
using Statistics
using GLPK

using Gurobi

# Variable preparation
In this section we reshape our first satge variables from matrices to column vectors

In [2]:
nScenarios = 243               # number of scenarios
nFactories = 3                 # number of factories
nPoints = 5                    # number of distribution points
nShip = nPoints * nFactories   # total number of shippement travels
nFirst = nFactories + nShip    # total number of first stage variables

prob = [0.25 0.50 0.25;        # probability
        0.25 0.50 0.25;
        0.25 0.50 0.25;
        0.30 0.40 0.30;
        0.30 0.40 0.30]

scenario = [150 160 170;       # scenarios
            100 120 135;
            250 270 300;
            300 325 350;
            600 700 800]

c2 = [2.49 5.21 3.76 4.85 2.07;# shipping cost
      1.46 2.54 1.83 1.86 4.76;
      3.26 3.08 2.60 3.76 4.45]

Cap = [500 450 650]            # capacity

1×3 Matrix{Int64}:
 500  450  650

We prefer to use one vector $x$ with `nFirst=18` components rather than two separate first stages variables $Prod$ and $Ship$. The first 3 coponents of $x$ are the $Prod$ variables while the 15 rest are the $Ship$ variables. it goes like this:
\begin{equation}
    x^T = [Prod_1,\, Prod_2,\, Prod_3,\, Ship_{1,1},\, Ship_{1,2},\dots,\, Ship_{2,1},\dots,\, Ship_{3,1},\dots,\, Ship_{3,5}]
\end{equation}

For that we create the $c$ matrix, where the first 3 components are the production price ($=14$) and the rest is the shipping cost.

In [3]:
# creating a c vector of size (1, 18)
c = [14.0, 14.0, 14.0]

for i = 1:nFactories
    for j = 1:nPoints
        push!(c, c2[i,j])
    end
end

We do the same thing for second satage variables. We consider one vector $y$ of 10 components rather than two variables $Sales$ and $Waste$. For that we need the $q$ vector, that contains the price and the waste cost.

In [4]:
# creating a q vector of size (1, 10)
q = [-24 -24 -24 -24 -24 4 4 4 4 4];

We need to construct all the possible scenarios. There are exactly $3^5=243$ scenarios. We construct a vector `scenario` containing each scenario and `prob` with the respective probability.

In [5]:
function make_prob_scenario(a, b, n, m, prob1, prob2, scenario1, scenario2)
    pp = []
    ss = []
    
    for i = 1:n
        for j = 1:m
            p = prob1[a,i] * prob2[b,j]
            push!(pp, p)
            # s = [scenario1[a,i], scenario2[b,j]]
            s = vcat(scenario1[a,i], scenario2[b,j])
            push!(ss, s)
        end
    end
    
    pp = reshape(pp, 1, n*m)
    ss = reshape(ss, 1, n*m)
    
    return pp, ss
end

make_prob_scenario (generic function with 1 method)

In [6]:
pp1, ss1 = make_prob_scenario(1, 2, nFactories, nFactories, prob, prob, scenario, scenario)
pp2, ss2 = make_prob_scenario(3, 4, nFactories, nFactories, prob, prob, scenario, scenario)
pp = vcat(pp1, pp2)
ss = vcat(ss1, ss2)
pp3, ss3 = make_prob_scenario(1, 2, 9, 9, pp, pp, ss, ss)
prob, scenario = make_prob_scenario(1, 5, 81, 3, pp3, prob, ss3, scenario)

println("probability and scenario done")

probability and scenario done


# 0- Before starting the exercice
Before starting the exercice, we want to see what answer we get with the `Gurobi.Optimizer`. We also want to compute the expetation value solution.

In [7]:
# import Pkg; Pkg.add("MathOptInterface")
MOI.INFEASIBLE

INFEASIBLE::TerminationStatusCode = 2

In [8]:
solver = Gurobi.Optimizer

Gurobi.Optimizer

In [9]:
m = Model()

@variable(m, x[1:nFirst] >= 0)
@variable(m, y[1:10, 1:nScenarios] >= 0)

for i = 1:nFactories
    @constraint(m, sum(x[3+5*(i-1) + j] for j in 1:5) == x[i])
    @constraint(m, x[i] <= Cap[i])
end

for j = 1:nPoints
    for s = 1:nScenarios
        @constraint(m, y[j, s] <= scenario[s][j])
        @constraint(m, sum(x[j+3+5*(i-1)] for i in 1:3) == y[j, s] + y[j+5, s])
    end
end

@objective(m, Min, sum(c[i]*x[i] for i in 1:nFirst) + sum(prob[s]*sum(q[j]*y[j, s] for j = 1:10) for s=1:nScenarios)
)

# println(m)
println("done")

done


In [10]:
set_optimizer(m, solver)
optimize!(m)

Academic license - for non-commercial use only - expires 2022-10-15
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2436 rows, 2448 columns and 7311 nonzeros
Model fingerprint: 0x295678b6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 8e+02]
Presolve removed 1218 rows and 0 columns
Presolve time: 0.00s
Presolved: 1218 rows, 2448 columns, 6093 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.7830000e+04   4.936875e+03   0.000000e+00      0s
     131   -1.0793000e+04   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 0s

       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase 

In [11]:
xGurobi = value.(x)

18-element Vector{Float64}:
 500.0
 450.0
 470.0
   0.0
   0.0
   0.0
   0.0
 500.0
 150.0
   0.0
   0.0
 300.0
   0.0
   0.0
 100.0
 270.0
   0.0
 100.0

In [12]:
objective_value(m)

-10792.999999999976

* expectation value solution

In [13]:
scenario_exp = [sum(prob[j]*scenario[j][i] for j=1:nScenarios) for i=1:5]
prob_exp = [1]

1-element Vector{Int64}:
 1

In [14]:
m = Model()

@variable(m, x[1:nFirst] >= 0)
@variable(m, y[1:10, 1:1] >= 0)

for i = 1:nFactories
    @constraint(m, sum(x[3+5*(i-1) + j] for j in 1:5) == x[i])
    @constraint(m, x[i] <= Cap[i])
end

for j = 1:nPoints
    for s = 1:1
        @constraint(m, y[j, s] <= scenario_exp[j])
        @constraint(m, sum(x[j+3+5*(i-1)] for i in 1:3) == y[j, s] + y[j+5, s])
    end
end

@objective(m, Min, sum(c[i]*x[i] for i in 1:nFirst) + sum(prob_exp[s]*sum(q[j]*y[j, s] for j = 1:10) for s=1:1)
)

# println(m)
println("done")

done


In [15]:
set_optimizer(m, solver)
optimize!(m)

Academic license - for non-commercial use only - expires 2022-10-15
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 28 columns and 51 nonzeros
Model fingerprint: 0x80a942dc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 7e+02]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 28 columns, 43 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.7830000e+04   1.970312e+02   0.000000e+00      0s
      12   -1.1862150e+04   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.00 seconds
Optimal objective -1.186215000e+04

User-callback calls 39, time in user-callback 0.00 sec


In [16]:
xexp = value.(x)

18-element Vector{Float64}:
 500.0
 450.0
 626.25
   0.0
   0.0
   0.0
   0.0
 500.0
 125.0
   0.0
   0.0
 325.0
   0.0
  34.99999999999994
 118.75
 272.5
   0.0
 200.0

In [17]:
objective_value(m)

-11862.150000000001

# 1-Multicut L-Shaped Algorithm
* We start by defining the master problem

In [18]:
function master()
    m = Model(with_optimizer(GLPK.Optimizer))
    
    @variable(m, x[1:nFirst] >= 0)
    @variable(m, θ[1:nScenarios] >= -1e8) # 
    
    # constraint
    for i = 1:nFactories
        @constraint(m, sum(x[3+5*(i-1) + j] for j in 1:5) == x[i])
        @constraint(m, x[i] <= Cap[i])
    end

    # Master problem objective
    @objective(m, Min, sum(c[i]*x[i] for i in 1:nFirst) 
        + sum(prob[i]*θ[i] for i in 1:nScenarios))

    return m, x, θ
end

master (generic function with 1 method)

In [19]:
# m, x, θ = master()
# println(m)

In [20]:
# optimize!(m)

In [21]:
# x = value.(x)

* Next we define the second stage problem

In [22]:
function secondStage(x, ξ)
    m = Model(with_optimizer(GLPK.Optimizer))
    # m = Model(solver = GurobiSolver())

    @variable(m, s[1:nPoints] >= 0)
    @variable(m, w[1:nPoints] >= 0)
    
    for j = 1:nPoints
        @constraint(m, s[j] <= ξ[j])
        @constraint(m, s[j] + w[j] == sum(x[3+j + 5*(i-1)] for i in 1:3))
    end
    
    @objective(m, Min, sum(q[j]*s[j] + q[j+5]*w[j] for j = 1:nPoints))
    
    status = optimize!(m)
    
    return m, status
end

secondStage (generic function with 1 method)

In [23]:
# x = [500; 450; 470; 0; 0; 0; 0; 500; 150; 0; 0; 300; 0; 0; 100; 270; 0; 100]
# ξ = scenario[1]
# m, status = secondStage(x, ξ)

In [24]:
# println(m)

In [25]:
# vaverage = objective_value(m)

* Then we define the dual problem

In [26]:
function secondStage_dual(x, ξ)
    
    mdual = Model(with_optimizer(GLPK.Optimizer))

    @variable(mdual, π[1:10])
    
    T1 = zeros(5, 18)
    T2 = [0 0 0 -1 0 0 0 0 -1 0 0 0 0 -1 0 0 0 0;
          0 0 0 0 -1 0 0 0 0 -1 0 0 0 0 -1 0 0 0;
          0 0 0 0 0 -1 0 0 0 0 -1 0 0 0 0 -1 0 0;
          0 0 0 0 0 0 -1 0 0 0 0 -1 0 0 0 0 -1 0;
          0 0 0 0 0 0 0 -1 0 0 0 0 -1 0 0 0 0 -1]
    T = vcat(T1, T2)
    h = [ξ[1]; ξ[2]; ξ[3]; ξ[4]; ξ[5]; 0; 0; 0; 0; 0]
    z = h - T*x
    
    for i = 1:nPoints
        @constraint(mdual, π[i] + π[5+i] <= q[i])
        @constraint(mdual, π[5+i] <= q[5+i])
        @constraint(mdual, π[i] <= 0)
    end
    
    @objective(mdual, Max, sum(z[i]*π[i] for i = 1:10))
    
    status = optimize!(mdual)
    
    E = transpose(value.(π))*T
    e = dot(value.(π), h)
    
    return mdual, status, π, E, e
end

secondStage_dual (generic function with 1 method)

In [27]:
# ξ = scenario[1]
# x = [1 for _ in 1:18]
# mdual, status, π, E, e = secondStage_dual(x, ξ)
# println(mdual)

In [28]:
# vaverage = objective_value(mdual)

In [29]:
# value.(π)

* We now define our stop function. This stop function is used both for the original Multicut method, where we have defined `Q` to be a vector composed of `nScenario=243` components (`Q` evaluated for each scenario) as you will see in the nest cell `lshaped()`. But we also used this method for the regularized `lshaped()`where `Q` in that case is but a scalar. That is why in the `stop(Q, θ, k)` function, we test wether `Q` is a scalar or not befor computing the stop condition.

In [30]:
function stop(Q, θ, k)
    nmax = 200
    tol = 1e-6
    
    if (k == nmax)
        return true 
    end
    
    # Q is a vector in case of normal multicut lshaped()
    if !(Q isa Float64)
        for i = 1:nScenarios
            if (( θ[i] < Q[i] - abs(tol*Q[i]) )|| k == 0)
                return false
            end
        end
    elseif ((sum(θ[i]*prob[i] for i in 1:nScenarios) < Q-abs(tol*Q)) || (k == nmax)) # Q is a scalar in the regularized lShaped()
        return false
    end
    
    return true
end

stop (generic function with 1 method)

* We define the multicut lshaped algorithm

In [31]:
function lshaped()
    
    k = 0       # iteration index
    nocuts = 0  # number of optimality cuts
    
    m, x, θ = master()
    n = 18

    Q = [+Inf for _ in 1:nScenarios]
    valθ = [-Inf for _ in 1:nScenarios]
    
    while (!stop(Q, valθ, k))  
        println(m)
        k += 1
            
        # solve master
        optimize!(m)
        status = termination_status(m)

        if (status != MOI.OPTIMAL)
            println("Error: status ", status)
            return status, m
        end

        xsol = value.(x)
        Q = []
        
        for i = 1:nScenarios
            Es = zeros(n)'
            Qs = 0.0
            
            # Solve the second-stage programs
            p = prob[i]
            mdual, status, π, Escen, escen = secondStage_dual(xsol, scenario[i])
            Qs = objective_value(mdual)
            Es = Escen
            push!(Q, Qs)
            
            # Build the optimality cut component
            valθ[i] = value.(θ)[i]
            if (valθ[i] < Qs)
                @constraint(m,  θ[i] >= (Qs - sum(Es[i]*(x[i] - xsol[i]) for i in 1:n)))
                nocuts += 1
            end
        end
        
        # println(Q)
        # println(valθ)
    end

    println("Solved in ", k, " iterations.")
    return x, m, k
end

lshaped (generic function with 1 method)

In [32]:
xls, m, klshaped = lshaped()

Min 14 x[1] + 14 x[2] + 14 x[3] + 2.49 x[4] + 5.21 x[5] + 3.76 x[6] + 4.85 x[7] + 2.07 x[8] + 1.46 x[9] + 2.54 x[10] + 1.83 x[11] + 1.86 x[12] + 4.76 x[13] + 3.26 x[14] + 3.08 x[15] + 2.6 x[16] + 3.76 x[17] + 4.45 x[18] + 0.00140625 θ[1] + 0.001875 θ[2] + 0.00140625 θ[3] + 0.001875 θ[4] + 0.0025000000000000005 θ[5] + 0.001875 θ[6] + 0.00140625 θ[7] + 0.001875 θ[8] + 0.00140625 θ[9] + 0.0028125 θ[10] + 0.00375 θ[11] + 0.0028125 θ[12] + 0.00375 θ[13] + 0.005000000000000001 θ[14] + 0.00375 θ[15] + 0.0028125 θ[16] + 0.00375 θ[17] + 0.0028125 θ[18] + 0.00140625 θ[19] + 0.001875 θ[20] + 0.00140625 θ[21] + 0.001875 θ[22] + 0.0025000000000000005 θ[23] + 0.001875 θ[24] + 0.00140625 θ[25] + 0.001875 θ[26] + 0.00140625 θ[27] + 0.0028125 θ[28] + 0.00375 θ[29] + 0.0028125 θ[30] + 0.00375 θ[31] + 0.005000000000000001 θ[32] + 0.00375 θ[33] + 0.0028125 θ[34] + 0.00375 θ[35] + 0.0028125 θ[36] + 0.005625 θ[37] + 0.0075 θ[38] + 0.005625 θ[39] + 0.0075 θ[40] + 0.010000000000000002 θ[41] + 0.0075 θ[42] + 0

Excessive output truncated after 524294 bytes.


 24 x[4] + 24 x[5] + 24 x[6] + 24 x[7] + 24 x[8] + 24 x[9] + 24 x[10] + 24 x[11] + 24 x[12] + 24 x[13] + 24 x[14] + 24 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[207] ≥ 0.0
 24 x[4] + 24 x[5] + 24 x[6] + 24 x[7] + 24 x[8] + 24 x[9] + 24 x[10] + 24 x[11] + 24 x[12] + 24 x[13] + 24 x[14] + 24 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[208] ≥ 0.0
 24 x[4] + 24 x[5] + 24 x[6] + 24 x[7] + 24 x[8] + 24 x[9] + 24 x[10] + 24 x[11] + 24 x[12] + 24 x[13] + 24 x[14] + 24 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[209] ≥ 0.0
 24 x[4] + 24 x[5] + 24 x[6] + 24 x[7] + 24 x[8] + 24 x[9] + 24 x[10] + 24 x[11] + 24 x[12] + 24 x[13] + 24 x[14] + 24 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[210] ≥ 0.0
 24 x[4] + 24 x[5] + 24 x[6] + 24 x[7] + 24 x[8] + 24 x[9] + 24 x[10] + 24 x[11] + 24 x[12] + 24 x[13] + 24 x[14] + 24 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[211] ≥ 0.0
 24 x[4] + 24 x[5] + 24 x[6] + 24 x[7] + 24 x[8] + 24 x[9] + 24 x[10] + 24 x[11] + 24 x[12] + 24 x[13] + 24 x[14] + 24 x[15] + 24 x[16

(VariableRef[x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14], x[15], x[16], x[17], x[18]], A JuMP Model
Minimization problem with:
Variables: 261
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1594 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 261 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, θ, 9)

In [33]:
xlshaped = value.(xls)

18-element Vector{Float64}:
 500.0
 450.0
 470.00000000113823
   0.0
   0.0
   0.0
   0.0
 500.0
 150.00000000271473
   0.0
   0.0
 299.9999999972853
   0.0
  -4.848790547204392e-10
 100.00000000124685
 269.99999999968355
   0.0
 100.00000000069268

In [34]:
println("Solved in ", klshaped, " iterations.")

Solved in 9 iterations.


# 2-Regularisation
We use the trust region regularization of our previous model

In [35]:
function firststage(x)
    return dot(x, c)
end

firststage (generic function with 1 method)

In [36]:
function lshaped(x0:: Vector)
    k = 0       # iteration index
    nocuts = 0  # number of optimality cuts
    
    m, x, θ = master()
    n = nFirst

    Q = 0.0
    valθ = [-Inf for _ in 1:nScenarios]
    
    success = false
    tractive = true

    # Compute the objective value at the initial solution
    fval0 = firststage(x0)
    for i = 1:nScenarios
        mdual, status, π, Escen, escen = secondStage_dual(x0, scenario[i])
        Qs = prob[i]*objective_value(mdual)
        Q += prob[i]*Qs
    end
    fval0 += Q
    optimize!(m)
    fcand0 = objective_value(m) - sum(value.(θ)[i]*prob[i] for i in 1:nScenarios) + Q
    
    Δ = max(0.1*maximum(x0),1)

    while (!stop(Q, valθ, k) || tractive)
        println(k, ". ", stop(Q, valθ, k), " ", tractive)
        
        # stop
        if stop(Q, valθ, k)
            tractive = false
            for i = 1:n
                set_lower_bound(x[i], 0.0)
                set_upper_bound(x[i], +Inf)
            end
        end
        
        # tractive
        if (tractive)
            for i = 1:n
                set_lower_bound(x[i], max(x0[i]-Δ, 0.0))
                set_upper_bound(x[i], x0[i]+Δ)
            end
        end

        println(m)
        k += 1

        optimize!(m)
        status = termination_status(m)
        
        if (status != MOI.OPTIMAL)
            println("Error: status ", status)
            return status, m
        end
        xsol = value.(x)
        
        Q = 0
        Qs = 0
        fval1 = firststage(xsol)
        fcand1 =  objective_value(m) - sum(prob[i]*valθ[i] for i in 1:nScenarios)
        
        for i = 1:nScenarios
            # Solve the second-stage programs
            p = prob[i]
            mdual, status, π, Es, es = secondStage_dual(xsol, scenario[i])
            Qs = objective_value(mdual)
            Q += prob[i]*Qs
            
            # We accept the solution and increase the trust-region if we have improved the objective value.
            # Otherwise, we shrink it.
            valθ[i] = value.(θ)[i]
            fcand_s = objective_value(m) - sum(prob[i]*valθ[i] for i in 1:nScenarios) + Qs
            fcand1 += prob[i]*Qs
            
            if (valθ[i] < Qs)
                @constraint(m,  θ[i] >= (Qs - sum(Es[i]*(x[i] - xsol[i]) for i in 1:n)))
                nocuts += 1
            end
        end
        
        fval1 += Q
        
        # if (fval < fcand) || !tractive
        if ((fval0 - fval1) > 0.8*(fcand0 - fcand1)) || !tractive
            x0 = value.(x)
            fval0 = fcand0
            fval1 = fcand1
            Δ *= 2
            success = true
        else
            Δ /= 2
            success = false
        end
        
    end
    
    println("Solved in ", k, " iterations.")
    return x, m, k
end

lshaped (generic function with 2 methods)

In [37]:
# ol = [500; 450; 470; 0; 0; 0; 0; 500; 150; 0; 0; 300; 0; 0; 100; 270; 0; 100]
x0 = [0 for _ in 1:nFirst]
xrg0, m, kregulazed = lshaped(x0)

0. false true
Min 14 x[1] + 14 x[2] + 14 x[3] + 2.49 x[4] + 5.21 x[5] + 3.76 x[6] + 4.85 x[7] + 2.07 x[8] + 1.46 x[9] + 2.54 x[10] + 1.83 x[11] + 1.86 x[12] + 4.76 x[13] + 3.26 x[14] + 3.08 x[15] + 2.6 x[16] + 3.76 x[17] + 4.45 x[18] + 0.00140625 θ[1] + 0.001875 θ[2] + 0.00140625 θ[3] + 0.001875 θ[4] + 0.0025000000000000005 θ[5] + 0.001875 θ[6] + 0.00140625 θ[7] + 0.001875 θ[8] + 0.00140625 θ[9] + 0.0028125 θ[10] + 0.00375 θ[11] + 0.0028125 θ[12] + 0.00375 θ[13] + 0.005000000000000001 θ[14] + 0.00375 θ[15] + 0.0028125 θ[16] + 0.00375 θ[17] + 0.0028125 θ[18] + 0.00140625 θ[19] + 0.001875 θ[20] + 0.00140625 θ[21] + 0.001875 θ[22] + 0.0025000000000000005 θ[23] + 0.001875 θ[24] + 0.00140625 θ[25] + 0.001875 θ[26] + 0.00140625 θ[27] + 0.0028125 θ[28] + 0.00375 θ[29] + 0.0028125 θ[30] + 0.00375 θ[31] + 0.005000000000000001 θ[32] + 0.00375 θ[33] + 0.0028125 θ[34] + 0.00375 θ[35] + 0.0028125 θ[36] + 0.005625 θ[37] + 0.0075 θ[38] + 0.005625 θ[39] + 0.0075 θ[40] + 0.010000000000000002 θ[41] + 0.

Excessive output truncated after 524449 bytes.

-4 x[4] - 4 x[5] + 24 x[6] + 24 x[7] + 24 x[8] - 4 x[9] - 4 x[10] + 24 x[11] + 24 x[12] + 24 x[13] - 4 x[14] - 4 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[219] ≥ -8540.000000000002
 -4 x[4] - 4 x[5] + 24 x[6] + 24 x[7] + 24 x[8] - 4 x[9] - 4 x[10] + 24 x[11] + 24 x[12] + 24 x[13] - 4 x[14] - 4 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[220] ≥ -8540.000000000002
 -4 x[4] - 4 x[5] + 24 x[6] + 24 x[7] + 24 x[8] - 4 x[9] - 4 x[10] + 24 x[11] + 24 x[12] + 24 x[13] - 4 x[14] - 4 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[221] ≥ -8540.000000000002
 -4 x[4] - 4 x[5] + 24 x[6] + 24 x[7] + 24 x[8] - 4 x[9] - 4 x[10] + 24 x[11] + 24 x[12] + 24 x[13] - 4 x[14] - 4 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[222] ≥ -8540.000000000002
 -4 x[4] - 4 x[5] + 24 x[6] + 24 x[7] + 24 x[8] - 4 x[9] - 4 x[10] + 24 x[11] + 24 x[12] + 24 x[13] - 4 x[14] - 4 x[15] + 24 x[16] + 24 x[17] + 24 x[18] + θ[223] ≥ -8540.000000000002
 -4 x[4] - 4 x[5] + 24 x[6] + 24 x[7] + 24 x[8] - 4 x[9] - 4 x[10] + 24 x[11] + 24 x[1

(VariableRef[x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14], x[15], x[16], x[17], x[18]], A JuMP Model
Minimization problem with:
Variables: 261
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1594 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 261 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 18 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, θ, 10)

In [38]:
xregulazed = value.(xrg0)

18-element Vector{Float64}:
 500.0
 450.0
 470.00000000113823
   0.0
   0.0
   0.0
   0.0
 500.0
 150.00000000271473
   0.0
   0.0
 299.9999999972853
   0.0
  -4.848790547204392e-10
 100.00000000124685
 269.99999999968355
   0.0
 100.00000000069268

In [39]:
println("Solved in ", kregulazed, " iterations.")

Solved in 10 iterations.


# 3-Discussion

The solution obtained by the lshaped method (and its regularized form) is (**approximatively**):
\begin{align*}
    Prod &= \begin{bmatrix}
                500 & 450 & 470
            \end{bmatrix}\\
    Ship &= \begin{bmatrix}
                0 & 0 & 0 & 0 & 500\\
                150 & 0 & 0 & 300 & 0\\
                0 & 100 & 270 & 0 & 100
            \end{bmatrix}           
\end{align*}
It is the same solution we obtained in section 0 using the Gurobi solver.

Its objective function value is:
\begin{equation}
    m \approx -10793
\end{equation}


We can see that the number of itterations in the multicut `lshaped()` method and its regularized form are quite the same:
* `knormal = 9`
* `kregulazed = 10`

However, the regularized itteration number should be expected to change when changing the initial decision candidate. When it is carefully chosen, the computation time can be significantly reduced. In the case of $x$ being a null vector, we got `kregulazed = 10`. However if replaced by the expectation value solution, which is 
\begin{align*}
    Prod &= \begin{bmatrix}
                500 & 450 & 626.25
            \end{bmatrix}\\
    Ship &= \begin{bmatrix}
                0 & 0 & 0 & 0 & 500\\
                125 & 0 & 0 & 325 & 0\\
                35 & 118.75 & 272.5 & 0 & 200
            \end{bmatrix}           
\end{align*}
we get `kregulazed = 8`.

One last observation. When starting our reguralized algorithm with the solution obtained in the normal multicut lshaped, i.e `xlshaped`, the itteration number stays the same (`kregulazed = 8`), however the errors in the $x$ value get reduced. For instance the 14th component of $x$, i.e $Ship_{3,1}$ moves from $ -4.848790547204392\cdot 10^{-9}$ to $2.48356476125385\cdot 10^{-11}$. That is a difference of order $10^{2}$. Other components also get their error reduced. 

We check this last remark using the following function:

In [40]:
function compare_error(a, b)
    error = zeros(nFirst)'
    for i in 1:nFirst
        error[i] = abs(a[i]-xGurobi[i])/abs(b[i]-xGurobi[i])
        if isnan(error[i])
            error[i] = 0
        end
    end
    println(error)
end

compare_error (generic function with 1 method)

For that, we start by generating `xreg_lshaped`, the solution obtained when starting with the optimal solution directly.

In [ ]:
# xreg_lshaped, _, _ = lshaped(xlshaped)

In [ ]:
# xreg_exp, _, _ = lshaped(xexp)

In [ ]:
# xreg_lshaped = value.(xreg_lshaped)
# xreg_exp = value.(xreg_exp)
# compare_error(xreg_lshaped, xreg_exp)